# Variational Quantum Eigensolver
We can use Variational Quantum Eigensolver for simulation of chemistry reaction.<br>
VQE is an algorithm that makes use of the variational method used in quantum mechanics.<br>
This means that we can estimate the ground state energy of a system<br>
by making an educated guess of its wave function, which represents the molecole.<br>
We then vary our wave function until we get minimum value of the ground state energy.<br>
VQE is a hybrid algorithm which consists of both quantum part and classical part.<br>
The quantum part computes the energy, and the classical part optimizes the variational parameter.<br>
In this notebook, we use VQE to compute the interatomic distance that gives the lowest energy.<br>


In [1]:
import numpy as np
import pylab
import copy
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.core import Hamiltonian, QubitMappingType

In [2]:
molecule = 'H .0 .0 -{0}; Li .0 .0 {0}'
distances = np.arange(0.5, 4.25, 0.25)
vqe_energies = []
hf_energies = []
exact_energies = []

In [3]:
for i, d in enumerate(distances):
    print('step', i)

    # set up experiment
    driver = PySCFDriver(molecule.format(d/2), basis='sto3g')
    qmolecule = driver.run()
    operator = Hamiltonian(qubit_mapping=QubitMappingType.PARITY,
                           two_qubit_reduction=True, freeze_core=True,
                           orbital_reduction=[-3, -2])
    qubit_op, aux_ops = operator.run(qmolecule)

    # exact classical result
    exact_result = NumPyMinimumEigensolver(qubit_op, aux_operators=aux_ops)
    exact_result = operator.process_algorithm_result(exact_result)

    # VQE
    optimizer = SLSQP(maxiter=1000)
    initial_state = HartreeFock(operator.molecule_info['num_orbitals'],
                                operator.molecule_info['num_particles'],
                                qubit_mapping=operator._qubit_mapping,
                                two_qubit_reduction=operator._two_qubit_reduction)
    
    var_form = UCCSD(num_orbitals=operator.molecule_info['num_orbitals'],
                     num_particles=operator.molecule_info['num_particles'],
                     initial_state=initial_state,
                     qubit_mapping=operator._qubit_mapping,
                     two_qubit_reduction=operator._two_qubit_reduction)
    
    algo = VQE(qubit_op, var_form, optimizer, aux_operators=aux_ops)

    vqe_result = algo.run(QuantumInstance(BasicAer.get_backend('statevector_simulator')))
    vqe_result = operator.process_algorithm_result(vqe_result)

    exact_energies.append(exact_result.energy)
    vqe_energies.append(vqe_result.energy)
    hf_energies.append(vqe_result.hartree_fock_energy)

step 0
/home/dohun/anaconda3/envs/qiskit/lib/python3.9/site-packages/qiskit/chemistry/core/hamiltonian.py:91: DeprecationWarning: The ChemistryOperator is deprecated as of Qiskit Aqua 0.8.0 and will be removed no earlier than 3 months after the release date. Instead, the FermionicTransformation can be used to transform QMolecules and construct ground state result objects.
  super().__init__()
/home/dohun/anaconda3/envs/qiskit/lib/python3.9/site-packages/qiskit/chemistry/core/hamiltonian.py:415: DeprecationWarning: Processing a dictionary result is deprecated, pass a (minimum) eigensolver result now.
  warnings.warn('Processing a dictionary result is deprecated,'


TypeError: 'NumPyMinimumEigensolver' object is not subscriptable